In [11]:
# Softmax example in TF using the classical Iris dataset
# Download iris.data from https://archive.ics.uci.edu/ml/datasets/Iris

import tensorflow as tf
import os

# this time weights form a matrix, not a column vector, one "weight vector" per class.
W = tf.Variable(tf.zeros([4, 3]), name="weights")
# so do the biases, one per class.
b = tf.Variable(tf.zeros([3], name="bias"))


def combine_inputs(X):
    return tf.matmul(X, W) + b


def inference(X):
    return tf.nn.softmax(combine_inputs(X))


def loss(X, Y):
    '''
    https://www.tensorflow.org/versions/r0.12/api_docs/python/nn/classification#sparse_softmax_cross_entropy_with_logits
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels, name=None)
    
    https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits
    sparse_softmax_cross_entropy_with_logits(
    _sentinel=None,
    labels=None,
    logits=None,
    name=None
    )
    '''
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=combine_inputs(X), labels=Y))


def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)])

    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)

    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)

    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)


def inputs():

    '''
    http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
    
    5.1,3.5,1.4,0.2,Iris-setosa
    4.9,3.0,1.4,0.2,Iris-setosa

    '''
    sepal_length, sepal_width, petal_length, petal_width, label =\
        read_csv(100, "iris.data", [[0.0], [0.0], [0.0], [0.0], [""]])

    # convert class names to a 0 based class index.
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.stack([
        tf.equal(label, ["Iris-setosa"]),
        tf.equal(label, ["Iris-versicolor"]),
        tf.equal(label, ["Iris-virginica"])
    ])), 0))

    # Pack all the features that we care about in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.stack([sepal_length, sepal_width, petal_length, petal_width]))

    return features, label_number


def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)


def evaluate(sess, X, Y):

    predicted = tf.cast(tf.argmax(inference(X), 1), tf.int32)

    print (sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))


# Launch the graph in a session, setup boilerplate
with tf.Session() as sess:

    tf.global_variables_initializer().run()

    X, Y = inputs()

    total_loss = loss(X, Y)
    train_op = train(total_loss)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    # actual training loop
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:
            print("loss: ", sess.run([total_loss]))

    evaluate(sess, X, Y)
    
    import time
    time.sleep(5)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

loss:  [1.0958562]
loss:  [1.017004]
loss:  [0.97047317]
loss:  [0.92646372]
loss:  [0.8612023]
loss:  [0.83758843]
loss:  [0.788728]
loss:  [0.7829538]
loss:  [0.78652793]
loss:  [0.71473837]
loss:  [0.70731413]
loss:  [0.71483195]
loss:  [0.67469811]
loss:  [0.67186683]
loss:  [0.64212656]
loss:  [0.61263323]
loss:  [0.67562795]
loss:  [0.63631523]
loss:  [0.63601011]
loss:  [0.62370592]
loss:  [0.55983549]
loss:  [0.57627547]
loss:  [0.55662084]
loss:  [0.57523942]
loss:  [0.57635665]
loss:  [0.51283801]
loss:  [0.55206907]
loss:  [0.51197916]
loss:  [0.48868752]
loss:  [0.57049632]
loss:  [0.5043937]
loss:  [0.50030172]
loss:  [0.54855394]
loss:  [0.47502121]
loss:  [0.54549348]
loss:  [0.48563087]
loss:  [0.44808847]
loss:  [0.5372501]
loss:  [0.46792611]
loss:  [0.52316236]
loss:  [0.50266272]
loss:  [0.44596621]
loss:  [0.49626696]
loss:  [0.44426468]
loss:  [0.42416346]
loss:  [0.49768692]
loss:  [0.41304162]
loss:  [0.49408597]
loss:  [0.45731044]
loss:  [0.4687584]
loss:  [0.